# Modeling a reach task with a Double Integrator in Tarski

In [1]:
import sys
sys.path = ['../../src'] + sys.path # this ensures the notebook uses the source version of Tarski

In [2]:
import tarski
from tarski.theories import Theory
from tarski.syntax import *
from tarski.io import rddl
from tarski.model import Model
from tarski.evaluators.simple import evaluate
from tarski.syntax.arithmetic import *
from tarski.syntax.arithmetic.special import *
from tarski.syntax.arithmetic.random import *

In [3]:
import numpy

In [4]:
lang = tarski.language('double_integrator', [Theory.EQUALITY, Theory.ARITHMETIC, Theory.SPECIAL, Theory.RANDOM])

In [5]:
vehicle = lang.sort('vehicle', lang.Object)

## States

In [6]:
# state variables
x = lang.function('x', vehicle, lang.Real)
y = lang.function('y', vehicle, lang.Real)
vx = lang.function('vx', vehicle, lang.Real)
vy = lang.function('vy', vehicle, lang.Real)

In [7]:
# control variables
ux = lang.function('ux', vehicle, lang.Real)
uy = lang.function('uy', vehicle, lang.Real)

In [8]:
# object
v001 = lang.constant('v001', vehicle)

## Task

In [9]:
# target
gx = lang.function('gx', lang.Real)
gy = lang.function('gy', lang.Real)

## Constraints

### Dynamics

In [10]:
# logical variable
v = lang.variable('v', vehicle)

In [11]:
dt = lang.function('dt', lang.Real)

In [12]:
A = lang.matrix([[0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]], lang.Real)

In [13]:
print(A)

[[0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]]


In [14]:
print(A.shape)

(4, 4)


In [15]:
x = lang.vector([x(v), y(v), vx(v), vy(v)], lang.Real)

In [16]:
print(x)

[[x(v/vehicle)]
 [y(v/vehicle)]
 [vx(v/vehicle)]
 [vy(v/vehicle)]]


In [17]:
A * x

*([[0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]], [[x(v/vehicle)]
 [y(v/vehicle)]
 [vx(v/vehicle)]
 [vy(v/vehicle)]])

In [19]:
B = lang.matrix([[0, 0], [0, 0], [1, 0], [0, 1]], lang.Real)

In [20]:
print(B)

[[0 0]
 [0 0]
 [1 0]
 [0 1]]


In [21]:
u = lang.vector([ux(v), uy(v)], lang.Real)

In [22]:
B * u

*([[0 0]
 [0 0]
 [1 0]
 [0 1]], [[ux(v/vehicle)]
 [uy(v/vehicle)]])

In [23]:
A * x + dt() * (B * u)

+(*([[0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]], [[x(v/vehicle)]
 [y(v/vehicle)]
 [vx(v/vehicle)]
 [vy(v/vehicle)]]), *(dt(), *([[0 0]
 [0 0]
 [1 0]
 [0 1]], [[ux(v/vehicle)]
 [uy(v/vehicle)]])))